In [1]:
%matplotlib inline

# レビューを機械学習できる形にする

## gensim版

1. 単語辞書をつくる
2. 全レビューを分かち書きし、ストップ・ワード処理する
3. 全レビューからコーパスをつくる
4. 各レビューを行列変換する

## HashingTrick版

In [2]:
import re
import gensim
import MeCab
import pandas as pd

### レビュー (tsv) を読み込む

In [3]:
csv = pd.read_csv( '~/work/sake/review.tsv', delimiter='\t' )

In [4]:
csv[:5]

,code,meigara,review,datetime
0,4,鰊御殿 山田錦,4合瓶900円で安売りだったので買ってみました。あまり期待してなかったのですが、わりと普通に...,2014-09-14 10:32:46
1,4,北の誉 鰊御殿 純米大吟醸酒,山田錦100％使用。 フルーティな香りと、あっさりした呑み口、 コクのある味わい。 文句の...,2014-05-23 21:35:40
2,4,純米米しぼり,アルコール度数14-15。精米歩合70％。 香りマイルド、深いコク。 オーソドックスな味わ...,2013-08-28 23:19:33
3,4,白ラベル,安いが美味いと思う、ぬる燗か常温がいいね、吟醸酒をブレンドして造ってるて噂があるけどどうでし...,2009-07-07 00:43:14
4,4,純米吟醸『涼しぼり』,近所の酒屋で購入。 「夏のにごり酒」という謳い文句通り、爽やかで呑みやすい。 香りもいいけど...,2008-06-16 23:37:21


In [5]:
mecab = MeCab.Tagger("-Ochasen")

### 1. 単語辞書を作成する

In [74]:
single = r"^[0-9０-９ぁ-んァ-ン！？・　!-/:-@≠\[-`{-~\u3001-\u303F]$"
pair    = r"^[ぁ-んァ-ン]{2}$"

In [75]:
def wakachi( s ):
    res = mecab.parse( s )
    lines = res.split( '\n' )
    words = [ line.split( '\t' )[0] for line in lines ]
    words.remove( 'EOS' )
    words.remove( '' )
    return words

In [76]:
def stop_words_set( ws ):
    return set ( w for w in ws if re.match( single, w ) is None and re.match( pair, w ) is None )

In [77]:
def stop_words( ws ):
    return [ w for w in ws if re.match( single, w ) is None and re.match( pair, w ) is None ]

In [78]:
csv['words'] = csv.apply( lambda x: wakachi( x['review'] ), axis=1 )

In [79]:
csv['tokens'] = csv.apply( lambda x: stop_words_set( x['words'] ), axis=1 )

In [80]:
csv[:5]

,code,meigara,review,datetime,words,tokens,tokens2,wc
0,4,鰊御殿 山田錦,4合瓶900円で安売りだったので買ってみました。あまり期待してなかったのですが、わりと普通に...,2014-09-14 10:32:46,"[4, 合, 瓶, 900, 円, で, 安売り, だっ, た, ので, 買っ, て, み,...","{なかっ, 普通, 特徴, 感, 900, 香, 安売り, アルコール, ほどよく, 円, ...","[合, 瓶, 900, 円, 安売り, 買っ, あまり, 期待, なかっ, 普通, 旨かっ,...","[合, 瓶, 900, 円, 安売り, 買っ, あまり, 期待, なかっ, 普通, 旨かっ,..."
1,4,北の誉 鰊御殿 純米大吟醸酒,山田錦100％使用。 フルーティな香りと、あっさりした呑み口、 コクのある味わい。 文句の...,2014-05-23 21:35:40,"[山田, 錦, 100, ％, 使用, 。, フルーティ, な, 香り, と, 、, あっさ...","{使用, 香り, 文句, 酒, 100, 大, 吟醸, フルーティ, あっさり, 高い, 口...","[山田, 錦, 100, ％, 使用, フルーティ, 香り, あっさり, 呑み, 口, 味わ...","[山田, 錦, 100, ％, 使用, フルーティ, 香り, あっさり, 呑み, 口, 味わ..."
2,4,純米米しぼり,アルコール度数14-15。精米歩合70％。 香りマイルド、深いコク。 オーソドックスな味わ...,2013-08-28 23:19:33,"[アルコール, 度数, 14, -, 15, 。, 精米, 歩合, 70, ％, 。, 香り...","{選手, 合う, アルコール, 精米, 苦味, 歩合, オーソドックス, 15, 14, 香...","[アルコール, 度数, 14, 15, 精米, 歩合, 70, ％, 香り, マイルド, 深...","[アルコール, 度数, 14, 15, 精米, 歩合, 70, ％, 香り, マイルド, 深..."
3,4,白ラベル,安いが美味いと思う、ぬる燗か常温がいいね、吟醸酒をブレンドして造ってるて噂があるけどどうでし...,2009-07-07 00:43:14,"[安い, が, 美味い, と, 思う, 、, ぬる, 燗, か, 常温, が, いい, ね,...","{燗, 酒, 思う, 吟醸, 美味い, 安い, 造っ, 常温, 噂, でしょ, ブレンド}","[安い, 美味い, 思う, 燗, 常温, 吟醸, 酒, ブレンド, 造っ, 噂, でしょ]","[安い, 美味い, 思う, 燗, 常温, 吟醸, 酒, ブレンド, 造っ, 噂, でしょ]"
4,4,純米吟醸『涼しぼり』,近所の酒屋で購入。 「夏のにごり酒」という謳い文句通り、爽やかで呑みやすい。 香りもいいけど...,2008-06-16 23:37:21,"[近所, の, 酒屋, で, 購入, 。, 「, 夏, の, にごり酒, 」, という, 謳...","{近所, 香り, 夏, なかっ, 爽やか, 通り, やすい, 謳い文句, 購入, にごり酒,...","[近所, 酒屋, 購入, 夏, にごり酒, という, 謳い文句, 通り, 爽やか, 呑み, ...","[近所, 酒屋, 購入, 夏, にごり酒, という, 謳い文句, 通り, 爽やか, 呑み, ..."


In [81]:
from functools import reduce

In [82]:
wdict = reduce( lambda s, x: s | x, list(csv.tokens) )
#wdict = reduce( lambda a, x: a + x, [1,2,3,4,5] )
len(wdict)

12336

In [83]:
csv['wc'] = csv.apply( lambda x: stop_words( x['words'] ), axis=1 )

In [84]:
def wc( a, items ):
    for x in items:
        if x not in a:
            a[x] = 1
        else:
            a[x] += 1
    return a

In [85]:
wc = reduce( lambda a, x: wc( a, x ), list(csv.wc), dict() )

In [86]:
wdf = pd.DataFrame( list(wc.items()), columns=['word', 'count'] )

In [87]:
sorted_wdf = wdf.sort_values( 'count', ascending=False )

In [89]:
sorted_wdf[:20]

,word,count
7190,酒,5995
1512,米,2148
8807,感じ,2039
5267,香り,2005
11092,味,1918
9614,飲み,1498
11761,純,1276
1228,吟醸,1267
2857,飲ん,1103
10961,味わい,1011
